**Train Test and Cross Validation**

In [2]:
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, f1_score 
from sklearn.neighbors import KNeighborsClassifier



In [3]:
# Load training data
data_path = '/workspaces/Final-Year-Project/Cleaned Data/TrainTestData.csv'
df = pd.read_csv(data_path)

# Drop missing values
df = df.dropna()

# Features and target variable
X = df.drop('Diabetes Status', axis=1)
y = df['Diabetes Status']


In [4]:
# Feature scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [5]:
# KNeighborsClassifier setup
knn = KNeighborsClassifier()

# Hyperparameter grid for GridSearchCV
param_grid = {
    'n_neighbors': [5, 7, 9, 11, 13, 15, 17, 19],  # Different values for k
}

# KFold cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# GridSearchCV setup with accuracy as the scoring parameter
grid_search = GridSearchCV(knn, param_grid, scoring='accuracy', cv=kf, n_jobs=-1, verbose=1)


In [6]:
# Fit GridSearchCV on the training data
grid_search.fit(X, y)

# Best KNN model from GridSearchCV
best_knn = grid_search.best_estimator_

#print best parameters
print(grid_search.best_params_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


{'n_neighbors': 15}


In [7]:
# Predictions on training data
y_pred = best_knn.predict(X)

# Print the best hyperparameters and the best score from GridSearchCV
print("Best Parameters for KNN:", grid_search.best_params_)
print("Mean Accuracy with Best Parameters:", grid_search.best_score_)

# Classification report on the training data
print("KNN Classification Report on Training Data:")
print(classification_report(y, y_pred))

# Confusion matrix on training data
print("KNN - Confusion Matrix on Training Data:")
print(confusion_matrix(y, y_pred))


Best Parameters for KNN: {'n_neighbors': 15}
Mean Accuracy with Best Parameters: 0.7322552697266438
KNN Classification Report on Training Data:
              precision    recall  f1-score   support

           0       0.80      0.70      0.75      1638
           1       0.73      0.82      0.78      1630

    accuracy                           0.76      3268
   macro avg       0.77      0.76      0.76      3268
weighted avg       0.77      0.76      0.76      3268

KNN - Confusion Matrix on Training Data:
[[1148  490]
 [ 287 1343]]


In [8]:
# Load validation data
valid_path = "/workspaces/Final-Year-Project/Cleaned Data/ValidationData.csv"
df_valid = pd.read_csv(valid_path)

# Drop any rows with missing values in the validation set
df_valid = df_valid.dropna()

# Prepare features and target for validation data
X_valid = df_valid.drop('Diabetes Status', axis=1)
y_valid = df_valid['Diabetes Status']

# Scale the validation data using the previously fitted scaler
X_valid_scaled = scaler.transform(X_valid)


In [9]:
# Predictions on validation data
y_pred_valid = best_knn.predict(X_valid_scaled)


In [10]:
# Print classification report for validation data
print(f"📌 KNN - Validation Classification Report:")
print(classification_report(y_valid, y_pred_valid))

# Print confusion matrix for validation data
print("🟦 KNN - Validation Confusion Matrix:")
print(confusion_matrix(y_valid, y_pred_valid))


📌 KNN - Validation Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.67      0.70       178
           1       0.71      0.77      0.74       186

    accuracy                           0.72       364
   macro avg       0.72      0.72      0.72       364
weighted avg       0.72      0.72      0.72       364

🟦 KNN - Validation Confusion Matrix:
[[120  58]
 [ 43 143]]


In [11]:
import pandas as pd
from sklearn.metrics import classification_report

# Assuming you already have y_valid (true labels) and y_pred_valid (predicted labels)
# Example classification report for AdaBoost (AB)
report = classification_report(y_valid, y_pred_valid, output_dict=True)

# Extract weighted scores
weighted_precision = report['weighted avg']['precision']
weighted_recall = report['weighted avg']['recall']
weighted_f1 = report['weighted avg']['f1-score']
accuracy = report['accuracy']

# Create a dictionary with the scores
ab_results = {
    'Model': 'K Nearest Nearest Neighbours(KNN)',
    'Precision': weighted_precision,
    'Recall': weighted_recall,
    'F1-Score': weighted_f1,
    'Accuracy': accuracy
}

# Convert the dictionary to a DataFrame
ab_df = pd.DataFrame([ab_results])

# Create a DataFrame to store results (if not already created)
results_df = pd.read_csv("/workspaces/Final-Year-Project/Results/model_results.csv")

# Concatenate the new row to the DataFrame
results_df = pd.concat([results_df, ab_df], ignore_index=True)

# Print the DataFrame to confirm it's added
print(results_df)

# Save the DataFrame to a CSV file
results_df.to_csv("/workspaces/Final-Year-Project/Results/model_results.csv", index=False)


                               Model  Precision    Recall  F1-Score  Accuracy
0                      AdaBoost (AB)   0.767212  0.752747  0.748389  0.752747
1                 Neural Network(NN)   0.762241  0.755495  0.753238  0.755495
2                  Decision Tree(DT)   0.748498  0.741758  0.739233  0.741758
3  K Nearest Nearest Neighbours(KNN)   0.723548  0.722527  0.721802  0.722527
4                 Neural Network(NN)   0.775001  0.769231  0.767458  0.769231
5                  Decision Tree(DT)   0.748498  0.741758  0.739233  0.741758
6  K Nearest Nearest Neighbours(KNN)   0.723548  0.722527  0.721802  0.722527


In [12]:
# Recalculate Macro average F1-score on validation data
validation_f1_macro = f1_score(y_valid, y_pred_valid, average='macro')
print(f"🏆 Macro F1-Score on Validation Data: {validation_f1_macro}")


🏆 Macro F1-Score on Validation Data: 0.7214152022854199
